In [1]:
%pip install azure-search-documents==11.6.0b9 azure-identity python-dotenv pandas jinja2 --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

AZURE_SEARCH_SERVICE_ENDPOINT=os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX=os.getenv("AZURE_SEARCH_INDEX_NAME")
AZURE_SEARCH_ADMIN_KEY=os.getenv("AZURE_SEARCH_ADMIN_KEY")

In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # toma las variables de entorno del archivo .env

endpoint = AZURE_SEARCH_SERVICE_ENDPOINT
index_name = AZURE_SEARCH_INDEX
credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)

In [3]:
from azure.search.documents import SearchClient
import pandas as pd

search_client = SearchClient(endpoint, index_name, credential)

def display_results(results):
    df = pd.json_normalize(list(results)).dropna(axis=1, how='all')
    df["chunk"] = df["chunk"].apply(lambda c: c[:300] + '...' if len(c) > 300 else c)
    first_cols = ['title', 'chunk', '@search.score']
    df = df[first_cols + [col for col in df.columns if col not in first_cols]]

    df = df.style.set_properties(**{
        'max-width': '500px',
        'text-align': 'left',
        'white-space': 'normal',
        'word-wrap': 'break-word'
    }).hide(axis="index")


    return df

In [4]:
results = search_client.search(search_text="What is Contoso", top=5, select=["title", "chunk"])

display_results(results)

title,chunk,@search.score
Northwind_Health_Plus_Benefits_Details.pdf,"potential side effects that may occur from these treatments, and to make sure that you are comfortable with the potential risks and benefits of the treatment. Chemotherapy And Radiation Therapy Chemotherapy and Radiation Therapy: Covered Services At Contoso, we are proud to offer our emplo...",4.381560
Northwind_Standard_Benefits_Details.pdf,"your coverage, please contact our customer service department at Contoso. Neurodevelopmental Therapy (Habilitation) Neurodevelopmental Therapy (Habilitation) At Contoso, we want to ensure that our employees and their dependents receive the best possible care and services. That’s why we’ve ...",4.030243
Northwind_Standard_Benefits_Details.pdf,"providers that are not available from participating providers. Additionally, in some cases, the health plan may cover non-participating providers’ charges if there are no participating providers in your area. Tips In order to avoid costly balance billing amounts, it is important to make sure...",3.696956
Northwind_Health_Plus_Benefits_Details.pdf,"the tips outlined above, you can help ensure that your request for services or treatments is approved in a timely manner and that you are receiving the most appropriate care. The Group And You OTHER INFORMATION ABOUT THIS PLAN The Group and You The Northwind Health Plus plan is a gro...",3.695086
Northwind_Health_Plus_Benefits_Details.pdf,"amounts. By understanding Northwind Health Plus's exclusions and following the tips outlined above, you can ensure that you are receiving the most comprehensive coverage available under the plan and avoid any unexpected costs. WHAT IF I HAVE OTHER COVERAGE? Coordinating Benefits With Othe...",3.382059


### Busqueda vectorial para acotar resultados

In [5]:
from azure.search.documents.models import VectorizableTextQuery

results = search_client.search(vector_queries=[VectorizableTextQuery(text="What is Contoso", k_nearest_neighbors=50, fields="text_vector")], top=5, select=["title", "chunk"])

display_results(results)

title,chunk,@search.score
MSFT_cloud_architecture_contoso.pdf,How a fictional but representative global organization has implemented the Microsoft Cloud Contoso in the Microsoft Cloud This topic is 1 of 7 in a series The Contoso Corporation Contoso s worldwide organization Elements of Contoso s implementation of the Microsoft cloud Networking Net...,0.852663
MSFT_cloud_architecture_contoso.pdf,mailto:cloudadopt@microsoft.com mailto:cloudadopt@microsoft.com https://azure.microsoft.com/services/expressroute/ https://azure.microsoft.com/services/expressroute/ https://azure.microsoft.com/services/expressroute/ https://azure.microsoft.com/services/expressroute/ https://aka.ms/o365protect_devic...,0.846799
MSFT_cloud_architecture_contoso.pdf,a fictional but representative global organization has implemented the Microsoft Cloud Contoso s app infrastructure Contoso has the following networking infrastructure. On-premises network WAN links connect the Paris headquarters to regional offices and regional offices to satellite office...,0.844584
MSFT_cloud_architecture_contoso.pdf,have identified the following elements when planning for the adoption of Microsoft s cloud offerings. Microsoft Cloud Identity for Enterprise Architects Microsoft Cloud Identity for Enterprise Architects Microsoft Cloud Networking for Enterprise Architects Microsoft Cloud Networking for...,0.838508
MSFT_cloud_architecture_contoso.pdf,https://technet.microsoft.com/library/mt775341.aspx https://technet.microsoft.com/library/mt775341.aspx How a fictional but representative global organization has implemented the Microsoft Cloud Contoso in the Microsoft Cloud Contoso s IT infrastructure and needs Mapping Contoso s busin...,0.837523


In [6]:
results = search_client.search(
    search_text="What is Contoso",
    vector_queries=[VectorizableTextQuery(text="What is Contoso", k_nearest_neighbors=50, fields="text_vector")],
    top=5,
    select=["title", "chunk"]
)

display_results(results)

title,chunk,@search.score
MSFT_cloud_architecture_contoso.pdf,"and tenants for Microsoft s cloud offeringsSubscriptions, licenses, accounts, and tenants for Microsoft s cloud offerings Tenants: This topic is 5 of 7 in a series Sales.Production Admin.Production IT.Development IT.Testing IT.Production Sales.Production IT.Production Sales.Production IT....",0.027526
MSFT_cloud_architecture_contoso.pdf,"scenarios How a fictional but representative global organization has implemented the Microsoft Cloud 1 2 3 4 5This topic is 7 of 7 in a series 6 With the networking, identity, and security infrastructure in place, Contoso began to address its business needs with enterprise cloud scenarios. ...",0.026743
MSFT_cloud_architecture_contoso.pdf,"built into service pricing, but in some cases you will need to purchase additional software licenses. User accounts User accounts are stored in an Azure AD tenant and can be synchronized from an on-premises identity provider such as Windows Server AD. Organization The Contoso Corpora...",0.025649
role_library.pdf,"be responsible for providing excellent customer service to all Contoso customers, addressing inquiries and resolving customer issues. Responsibilities: • Maintain a professional and courteous attitude when interacting with customers via phone, email, and chat • Respond to custome...",0.024052
MSFT_cloud_architecture_contoso.pdf,mailto:cloudadopt@microsoft.com mailto:cloudadopt@microsoft.com https://azure.microsoft.com/services/expressroute/ https://azure.microsoft.com/services/expressroute/ https://azure.microsoft.com/services/expressroute/ https://azure.microsoft.com/services/expressroute/ https://aka.ms/o365protect_devic...,0.023969
